In [13]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

C:\Users\macat\AppData\Local\Temp\ipykernel_18364\3777615979.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython.display
  from IPython.core.display import display, HTML


# Lab | Natural Language Processing
### SMS: SPAM or HAM

### Let's prepare the environment

In [14]:
import os
import re
import string
from pathlib import Path
import pandas as pd

- Read Data for the Fraudulent Email Kaggle Challenge
- Reduce the training set to speead up development. 

In [15]:
import pandas as pd

data = pd.read_csv("kg_train.csv", encoding="latin-1")
test = pd.read_csv("kg_test.csv",  encoding="latin-1")

print(data.shape, test.shape)
data.head()

(5964, 2) (5964, 1)


,text,label
0,"DEAR SIR, STRICTLY A PRIVATE BUSINESS PROPOSAL...",1
1,Will do.,0
2,Nora--Cheryl has emailed dozens of memos about...,0
3,Dear Sir=2FMadam=2C I know that this proposal ...,1
4,fyi,0


In [16]:
## Read Data for the Fraudulent Email Kaggle Challenge
data = pd.read_csv("kg_train.csv", encoding="latin-1")

# Reduce the training set to speed up development. 
# Modify for final system
data = data.head(1000)
print(data.shape)
data.fillna("",inplace=True)

(1000, 2)


### Let's divide the training and test set into two partitions

In [17]:
from sklearn.model_selection import train_test_split
X = data["text"]
y = data["label"]
X_train, X_val, y_train, y_val = train_test_split(
    X, y,
    test_size=0.2,        
    stratify=y,            
    random_state=42        
)

print("Train size:", len(X_train))
print("Validation size:", len(X_val))

Train size: 800
Validation size: 200


## Data Preprocessing

In [18]:
import string
from nltk.corpus import stopwords
print(string.punctuation)
print(stopwords.words("english")[100:110])
from nltk.stem.snowball import SnowballStemmer
snowball = SnowballStemmer('english')

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
['needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on']


In [19]:
def _find_file(fname: str):
    """Search for a file name in a few likely places. Returns Path or None."""
    candidates = [
        Path(fname),
        Path.cwd() / fname,
        Path("/mnt/data") / fname,
        Path("data") / fname,
        Path("dataset") / fname,
        Path("datasets") / fname,
    ]
    for p in candidates:
        if p.is_file():
            return p
    return None

def _read_csv_safely(fname: str, encoding="latin-1"):
    p = _find_file(fname)
    if p is None:
        return None, None
    try:
        df = pd.read_csv(p, encoding=encoding)
        return df, str(p)
    except Exception as e:
        return None, f"Found at {p} but failed to read: {e}"

In [20]:
data, data_path_info = _read_csv_safely("kg_train.csv")
test, test_path_info = _read_csv_safely("kg_test.csv")
TEXT_COL = "text"
LABEL_COL = "label"
if data is None:
    # Tiny fallback so the notebook still runs.
    # Two "spam-ish" examples and two "ham" (legit) examples.
    data = pd.DataFrame({
        TEXT_COL: [
            "CONGRATS!!! You won $1000. Click here to claim your prize now!!!",
            "<html><body>Dear user, your account needs verification. Send your password.</body></html>",
            "Hi team, here are the minutes from today's meeting attached.",
            "Lunch tomorrow at 12? Let's meet by the cafeteria."
        ],
        LABEL_COL: [1, 1, 0, 0],
    })
    data_path_info = "FAKE (in-notebook) sample data used because kg_train.csv was not found."
else:
    # Ensure the expected columns exist; if the original CSV uses other names, try to map them.
    lowered_cols = {c.lower(): c for c in data.columns}
    if TEXT_COL not in data.columns:
        # Best guess mappings
        for guess in ["email_text", "message", "content", "body", "text_"]:
            if guess in lowered_cols:
                data.rename(columns={lowered_cols[guess]: TEXT_COL}, inplace=True)
                break
    if LABEL_COL not in data.columns:
        for guess in ["label", "spam", "target", "class", "is_spam"]:
            if guess in lowered_cols:
                data.rename(columns={lowered_cols[guess]: LABEL_COL}, inplace=True)
                break
    # If still missing, raise a friendly error
    if TEXT_COL not in data.columns or LABEL_COL not in data.columns:
        raise ValueError(f"Your training CSV must have '{TEXT_COL}' and '{LABEL_COL}' columns (or recognizable equivalents). "
                         f"Present columns: {list(data.columns)}")

if test is None:
    test = pd.DataFrame({TEXT_COL: ["This is a placeholder test email."], LABEL_COL: [0]})
    test_path_info = "FAKE (in-notebook) sample test data used because kg_test.csv was not found."

print(f"[INFO] Training data source: {data_path_info}")
print(f"[INFO] Test data source:     {test_path_info}")
print(f"[INFO] Train shape: {data.shape} | Test shape: {test.shape}")

[INFO] Training data source: kg_train.csv
[INFO] Test data source:     kg_test.csv
[INFO] Train shape: (5964, 2) | Test shape: (5964, 1)


## Now, we have to clean the html code removing words

- First we remove inline JavaScript/CSS
- Then we remove html comments. This has to be done before removing regular tags since comments can contain '>' characters
- Next we can remove the remaining tags

In [24]:
from bs4 import BeautifulSoup, Comment

def strip_html(raw_html: str) -> str:
    """Remove HTML tags, scripts, styles, and comments from a string."""
    if not isinstance(raw_html, str):
        return ""
    
    soup = BeautifulSoup(raw_html, "html.parser")
    
    # remove script and style
    for tag in soup(["script", "style"]):
        tag.decompose()
    
    # remove HTML comments
    for c in soup.find_all(string=lambda t: isinstance(t, Comment)):
        c.extract()
    
    # get visible text
    text = soup.get_text(separator=" ")
    return text.strip()

# --------------------------
# Apply to your dataframes
# --------------------------

def add_clean_text(df: pd.DataFrame, text_col: str) -> pd.DataFrame:
    df = df.copy()
    df["text_nohtml"] = df[text_col].fillna("").astype(str).map(strip_html)
    return df

# detect which text column exists
for candidate in ["text", "message", "body", "content"]:
    if candidate in data_train.columns:
        TEXT_COL = candidate
        break
else:
    raise ValueError(f"No suitable text column found in {list(data_train.columns)}")

# apply to train and val sets
data_train = add_clean_text(data_train, TEXT_COL)
data_val   = add_clean_text(data_val, TEXT_COL)

# quick check
data_train[["text_nohtml"]].head()

NameError: name 'data_train' is not defined

- Remove all the special characters
    
- Remove numbers
    
- Remove all single characters
 
- Remove single characters from the start

- Substitute multiple spaces with single space

- Remove prefixed 'b'

- Convert to Lowercase

In [ ]:
for df_ in (data, test):
    df_[TEXT_COL] = df_[TEXT_COL].fillna("").astype(str)
    if LABEL_COL in df_.columns:
        df_[LABEL_COL] = df_[LABEL_COL].fillna(0).astype(int)

## Now let's work on removing stopwords
Remove the stopwords.

In [ ]:
def get_stopwords():
    try:
        from nltk.corpus import stopwords
        try:
            return set(stopwords.words("english"))
        except LookupError:
            # Try downloading if missing
            import nltk
            nltk.download("stopwords", quiet=True)
            return set(stopwords.words("english"))
    except Exception:
        # Fallback minimal list
        return set("""a an and are as at be by for from has have he her hers him his i in is it its 
                      me my not of on or our ours she that the their them they this to was we were will with you your"""
                   .split())

STOP_WORDS = get_stopwords()
PUNCT_TABLE = str.maketrans("", "", string.punctuation)

def remove_stopwords(text: str) -> str:
    if not isinstance(text, str):
        return ""
    text = text.lower().translate(PUNCT_TABLE)
    tokens = re.sub(r"[^a-z\s]", " ", text).split()
    return " ".join([t for t in tokens if t not in STOP_WORDS])
sample = "This is just a simple example to show how stopword removal works!"
print("Before:", sample)
print("After:", remove_stopwords(sample))

Before: This is just a simple example to show how stopword removal works!
After: simple example show stopword removal works


## Tame Your Text with Lemmatization
Break sentences into words, then use lemmatization to reduce them to their base form (e.g., "running" becomes "run"). See how this creates cleaner data for analysis!

In [ ]:
def get_lemmatizer():
    try:
        from nltk.stem import WordNetLemmatizer
        import nltk
    
        try:
            nltk.data.find("corpora/wordnet")
        except LookupError:
            nltk.download("wordnet", quiet=True)
            nltk.download("omw-1.4", quiet=True)  # for more lemmas
        return WordNetLemmatizer().lemmatize
    except Exception:
        
        return lambda w: w

LEMMATIZE = get_lemmatizer()

In [ ]:
PUNCT_TABLE = str.maketrans("", "", string.punctuation)

def lemmatize_text(text: str) -> str:
    if not isinstance(text, str):
        return ""
    # Lowercase + remove punctuation
    text = text.lower().translate(PUNCT_TABLE)
    # Keep only letters and spaces
    tokens = re.sub(r"[^a-z\s]", " ", text).split()
    # Lemmatize each token
    return " ".join([LEMMATIZE(t) for t in tokens])
sample = "The cats are running faster than the dogs"
print("Before:", sample)
print("After:", lemmatize_text(sample))

Before: The cats are running faster than the dogs
After: the cat are running faster than the dog


## Bag Of Words
Let's get the 10 top words in ham and spam messages (**EXPLORATORY DATA ANALYSIS**)

In [ ]:
from collections import Counter
DF = None
for df_name in ["data_train", "train_df", "data", "df", "dataset"]:
    if df_name in globals():
        cand = globals()[df_name]
        if isinstance(cand, pd.DataFrame) and len(cand.columns) > 0:
            DF = cand
            print(f"[INFO] Using dataframe: {df_name} with columns: {list(DF.columns)}")
            break

if DF is None:
    raise ValueError("Couldn't find a dataframe. Expected one of: data_train, train_df, data, df, dataset.")

TEXT_CANDIDATES  = [
    "preprocessed_text", "text_nohtml", "text",
    "message", "email_text", "content", "body", "sms", "message_text"
]
LABEL_CANDIDATES = ["label", "target", "spam", "class", "is_spam", "Category", "category", "Label"]

TEXT_COL = next((c for c in TEXT_CANDIDATES  if c in DF.columns), None)
LABEL_COL = next((c for c in LABEL_CANDIDATES if c in DF.columns), None)

if TEXT_COL is None:
    raise ValueError(f"No text column found. Looked for: {TEXT_CANDIDATES}. "
                     f"Available columns: {list(DF.columns)}")

if LABEL_COL is None:
    raise ValueError(f"No label column found. Looked for: {LABEL_CANDIDATES}. "
                     f"Available columns: {list(DF.columns)}")

print(f"[INFO] Using TEXT_COL='{TEXT_COL}', LABEL_COL='{LABEL_COL}'")

labels = DF[LABEL_COL]

# If labels are already numeric binary (0/1), keep them.
if labels.dtype.kind in "iu" and set(labels.unique()) <= {0,1}:
    y = labels.astype(int)
else:
    # Convert common string labels
    mapping = {}
    uniq = set(str(v).strip().lower() for v in labels.unique())
    if "spam" in uniq and "ham" in uniq:
        mapping = {"spam": 1, "ham": 0}
    elif "spam" in uniq and "not spam" in uniq:
        mapping = {"spam": 1, "not spam": 0}
    elif "pos" in uniq and "neg" in uniq:
        mapping = {"pos": 1, "neg": 0}
    else:
        # Fallback: try to infer by name of LABEL_COL
        # If column name suggests spam, treat nonzero/True as spam
        y = labels.astype(str).str.lower().isin(["1","true","spam"]).astype(int)
        print("[WARN] Unrecognized label values. Interpreting '1/true/spam' as spam.")
    if mapping:
        y = labels.astype(str).str.strip().str.lower().map(mapping)
        if y.isna().any():
            raise ValueError(f"Some labels couldn't be mapped with {mapping}. Unique labels: {uniq}")

def tokenize(text):
    # Lowercase, keep only letters, split by whitespace
    return re.findall(r"[a-zA-Z]+", str(text).lower())

ham_texts  = DF.loc[y == 0, TEXT_COL].fillna("").tolist()
spam_texts = DF.loc[y == 1, TEXT_COL].fillna("").tolist()

ham_counter  = Counter()
spam_counter = Counter()

for msg in ham_texts:
    ham_counter.update(tokenize(msg))
for msg in spam_texts:
    spam_counter.update(tokenize(msg))

top_ham  = ham_counter.most_common(10)
top_spam = spam_counter.most_common(10)

print("\nTop 10 words in HAM messages:")
for word, count in top_ham:
    print(f"{word:<18} {count}")

print("\nTop 10 words in SPAM messages:")
for word, count in top_spam:
    print(f"{word:<18} {count}")

[INFO] Using dataframe: train_df with columns: ['text', 'label', 'text_nohtml', 'text_clean']
[INFO] Using TEXT_COL='text_nohtml', LABEL_COL='label'

Top 10 words in HAM messages:
the                1602
to                 929
and                737
of                 694
a                  570
in                 548
that               379
s                  341
for                340
is                 336

Top 10 words in SPAM messages:
the                5633
to                 4359
of                 3973
and                3138
i                  2859
in                 2604
you                2577
a                  2387
e                  2266
this               2082


## Extra features

In [ ]:
def pick_text_col(df: pd.DataFrame):
    for c in ["preprocessed_text", "text_nohtml", "text", "message", "body", "content"]:
        if c in df.columns:
            return c
    raise ValueError(f"No suitable text column found in: {list(df.columns)}")

def pick_raw_col(df: pd.DataFrame, processed_col: str):
    # Prefer a less-processed version for uppercase and punctuation stats
    for c in ["text_nohtml", "text", "message", "body", "content"]:
        if c in df.columns and c != processed_col:
            return c
    return processed_col

money_simbol_list = "|".join(["euro","dollar","pound","€",r"\$"])
suspicious_words = "|".join(["free","cheap","sex","money","account","bank","fund","transfer","transaction","win","deposit","password"])

data_train['money_mark'] = data_train['preprocessed_text'].str.contains(money_simbol_list)*1
data_train['suspicious_words'] = data_train['preprocessed_text'].str.contains(suspicious_words)*1
data_train['text_len'] = data_train['preprocessed_text'].apply(lambda x: len(x)) 

data_val['money_mark'] = data_val['preprocessed_text'].str.contains(money_simbol_list)*1
data_val['suspicious_words'] = data_val['preprocessed_text'].str.contains(suspicious_words)*1
data_val['text_len'] = data_val['preprocessed_text'].apply(lambda x: len(x)) 

data_train.head()

NameError: name 'pd' is not defined

## How would work the Bag of Words with Count Vectorizer concept?

In [ ]:
# Your code

## TF-IDF

- Load the vectorizer

- Vectorize all dataset

- print the shape of the vetorized dataset

In [ ]:
# Your code

## And the Train a Classifier?

In [ ]:
# Your code

### Extra Task - Implement a SPAM/HAM classifier

https://www.kaggle.com/t/b384e34013d54d238490103bc3c360ce

The classifier can not be changed!!! It must be the MultinimialNB with default parameters!

Your task is to **find the most relevant features**.

For example, you can test the following options and check which of them performs better:
- Using "Bag of Words" only
- Using "TF-IDF" only
- Bag of Words + extra flags (money_mark, suspicious_words, text_len)
- TF-IDF + extra flags


You can work with teams of two persons (recommended).

In [ ]:
# Your code